# Examining SFI's Funding Portfolio
*by Neil Tynan @ATU*

![Science foundation Ireland](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fwww.sfi.ie%2Fsfi-logo-and-guidelines%2FSFI_logo_2016__master_colour_Boarder.jpg&f=1&nofb=1&ipt=00fae12be8242a03daccb98a3f01f7dd576cb3e81cc0557cd78526531ffdf107&ipo=images)
###### *“Logo provided by @science_foundation_ireland”*

This assignment looks at building a pie chart to examine the email domains of users in a sample file.

## Import Libraries
---

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime